In [170]:
import pandas as pd
import datetime

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [171]:
merged1 = pd.read_csv('new_data/merged_clean.csv', index_col=0)
print(merged1.shape)
merged2 = pd.read_csv('new_data/merged_clean_2.csv', index_col=0)
print(merged2.shape)
merged = merged1.append(merged2)
print(merged.shape)

(271100, 21)
(207447, 21)
(478547, 21)


/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_31525/3283179314.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged = merged1.append(merged2)


In [172]:
merged.drop_duplicates(inplace=True)
merged.dropna(subset=['ProfileScore:', 'Vert. meters:', 'Distance:'], how='all', inplace=True)
merged.shape

(388692, 21)

In [173]:
merged.columns

Index(['name', 'year', 'type', 'date', 'result', 'gc', 'icon', 'race_ref',
       'race_name', 'race_detail', 'race_rank', 'distance', 'points',
       'Avg. speed winner:', 'Distance:', 'Parcours type:', 'ProfileScore:',
       'Vert. meters:', 'Startlist quality score:', 'Won how: ',
       'adjusted_points'],
      dtype='object')

In [161]:
merged['ProfileScore:'].value_counts()

0.0      13766
1.0       3887
5.0       3050
6.0       2762
9.0       2522
         ...  
205.0        5
538.0        4
289.0        1
259.0        1
335.0        1
Name: ProfileScore:, Length: 403, dtype: int64

In [162]:
merged['date'] = pd.to_datetime(merged['date'])

### Linear regression for profile score

In [163]:
no_na = merged.dropna(subset=['ProfileScore:', 'Vert. meters:'], how='any')
print(no_na.shape)
no_na = no_na[no_na['ProfileScore:'] > 10]
print(no_na.shape)

(253924, 21)
(216812, 21)


In [164]:
# profile score via vert and distance
X  = no_na[['Distance:', 'Vert. meters:']]
y = no_na['ProfileScore:']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

tree = DecisionTreeRegressor()

tree.fit(X_train_scaled, y_train)

tree.score(X_test_scaled, y_test)

0.9997726014389591

In [169]:
profile_na = merged[merged['ProfileScore:'].isna()]
smaller_ten = merged[merged['ProfileScore:'] <= 10]
impute_profile = profile_na.append(smaller_ten)
impute_profile.dropna(subset=['Distance:', 'Vert. meters:'], how='any', inplace=True)

X_pred = impute_profile[['Distance:', 'Vert. meters:']]

X_pred_scaled = scaler.transform(X_pred)

impute_profile['predicted_score'] = tree.predict(X_pred_scaled)
impute_profile

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_31525/821046524.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  impute_profile = profile_na.append(smaller_ten)


,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points,predicted_score
3893,egan-bernal,2020,etappe,2020-08-01,6,8.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,30.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,52.0
3894,andrey-amador,2020,etappe,2020-08-01,77,77.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,52.0
3895,jonathan-castroviejo,2020,etappe,2020-08-01,75,75.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,52.0
3896,pavel-sivakov,2020,etappe,2020-08-01,40,41.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,52.0
3897,dylan-van-baarle,2020,etappe,2020-08-01,81,81.0,stage,race/la-route-d-occitanie/2020/stage-1,la-route-d-occitanie,Stage 1 - Saint-Affrique › Cazouls-lès-Béziers,...,0.0,41.35,189.5,p2,NaN,2346.0,496,? - let us know!,NaN,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198764,marcos-garcia,2010,etappe,2010-08-28,12,79.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,20.0
198765,vladimir-isaychev,2010,etappe,2010-08-28,12,75.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,20.0
198766,serafin-martinez,2010,etappe,2010-08-28,12,80.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,20.0
198767,gonzalo-rabunal-rios,2010,etappe,2010-08-28,12,81.0,chrono,race/vuelta-a-espana/2010/stage-1,vuelta-a-espana,Stage 1 (TTT) - Sevilla › Sevilla,...,18.0,55.32,13.0,p1,0.0,33.0,1132,Time Trial,0.0,20.0


In [ ]:
impute_profile = merged

### KNN to impute parcours type

In [101]:
merged['Parcours type:'].value_counts()

p2    107987
p1     87898
p5     57180
p0     57019
p3     50053
p4     28555
Name: Parcours type:, dtype: int64

In [80]:
knn = KNeighborsClassifier(n_neighbors=5)

px0 = merged[merged['Parcours type:']!='p0']

px0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)

X = px0[['ProfileScore:', 'Vert. meters:']]

y = px0['Parcours type:']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)

X_test_scaled = scaler.transform(X_test)

knn.fit(X_train_scaled, y_train)

knn.score(X_test_scaled, y_test)


/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_31525/3379125814.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  px0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)


0.9977640677404662

In [99]:
p0 = merged[merged['Parcours type:']=='p0']

#p0.dropna(subset = ['ProfileScore:', 'Vert. meters:'], how='any', inplace=True)

X_pred = p0[['ProfileScore:', 'Vert. meters:']]

X_pred_scaled = scaler.transform(X_pred)

labels = knn.predict(X_pred_scaled)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [100]:
p0

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,distance,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points
4112,egan-bernal,2020,one_day,2020-01-30,3,NaN,chrono,race/nc-colombia-itt/2020/result,nc-colombia-itt,National Championships Colombia - ITT (NC),...,41.4,40.0,51.71,41.4,p0,NaN,318.0,100,Time Trial,NaN
4113,daniel-felipe-martinez,2020,one_day,2020-01-30,1,NaN,chrono,race/nc-colombia-itt/2020/result,nc-colombia-itt,National Championships Colombia - ITT (NC),...,41.4,50.0,51.71,41.4,p0,NaN,318.0,100,Time Trial,NaN
4114,nairo-quintana,2020,one_day,2020-01-30,2,NaN,chrono,race/nc-colombia-itt/2020/result,nc-colombia-itt,National Championships Colombia - ITT (NC),...,41.4,44.0,51.71,41.4,p0,NaN,318.0,100,Time Trial,NaN
4115,winner-anacona,2020,one_day,2020-01-30,7,NaN,chrono,race/nc-colombia-itt/2020/result,nc-colombia-itt,National Championships Colombia - ITT (NC),...,41.4,28.0,51.71,41.4,p0,NaN,318.0,100,Time Trial,NaN
4116,johan-esteban-chaves,2020,one_day,2020-01-30,5,NaN,chrono,race/nc-colombia-itt/2020/result,nc-colombia-itt,National Championships Colombia - ITT (NC),...,41.4,32.0,51.71,41.4,p0,NaN,318.0,100,Time Trial,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207442,gustavo-rodriguez-iglesias,2010,etappe,2010-06-30,106,NaN,stage,race/course-cycliste-de-solidarnosc/2010/stage-2,course-cycliste-de-solidarnosc,Stage 2 - Radomsko › Belchatów,...,78.0,0.0,44.30,78.0,p0,NaN,NaN,65,? - let us know!,NaN
207443,ezequiel-mosquera-miguez,2010,etappe,2010-06-30,104,NaN,stage,race/course-cycliste-de-solidarnosc/2010/stage-1,course-cycliste-de-solidarnosc,Stage 1 - Lódz › Kleszczow,...,138.4,0.0,47.15,138.4,p0,NaN,NaN,65,? - let us know!,NaN
207444,gustavo-cesar-veloso,2010,etappe,2010-06-30,31,NaN,stage,race/course-cycliste-de-solidarnosc/2010/stage-1,course-cycliste-de-solidarnosc,Stage 1 - Lódz › Kleszczow,...,138.4,0.0,47.15,138.4,p0,NaN,NaN,65,? - let us know!,NaN
207445,marcos-garcia,2010,etappe,2010-06-30,34,NaN,stage,race/course-cycliste-de-solidarnosc/2010/stage-1,course-cycliste-de-solidarnosc,Stage 1 - Lódz › Kleszczow,...,138.4,0.0,47.15,138.4,p0,NaN,NaN,65,? - let us know!,NaN


In [90]:
p0['adjusted parcours type'] = labels

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_31525/2894937052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p0['adjusted parcours type'] = labels


In [92]:
merged.loc[p0.index, 'Parcours type:'] = p0['adjusted parcours type']

In [95]:
merged['Parcours type:'].value_counts()

p2    107987
p1     87898
p5     57180
p0     57019
p3     50053
p4     28555
Name: Parcours type:, dtype: int64

In [55]:
merged.groupby('Parcours type:')[['Vert. meters:', 'ProfileScore:']].mean()

,Vert. meters:,ProfileScore:
Parcours type:,,
p0,2349.370898,102.259215
p1,790.392832,16.004324
p2,2173.378585,85.287012
p3,2353.088346,109.706758
p4,3324.672194,183.478785
p5,3566.446305,281.490726


In [ ]:
parcours = {
    'p1' : 'fl',
    'p2' : 'hi_fl',
    'p3' : 'hi_hi',
    'p4' : 'mo_fl',
    'p5' : 'mo_mo'
}

In [53]:
merged[merged['Parcours type:']=='p0'].race_detail.value_counts()

World Championships - Road Race (WC)                                                    741
National Championships France - Road Race (NC)                                          641
National Championships Italy - Road Race (NC)                                           504
World Championships - TTT (WC)                                                          400
National Championships Belgium - Road Race (NC)                                         397
                                                                                       ... 
Stage 6 - Moca › Jarabacoa                                                                1
Stage 7 - La Vega › Santo Domingo                                                         1
Stage 8 (ITT) - Santo Domingo Mirador Sur › Santo Domingo Mirador Sur                     1
Stage 9 - San Cristóbal › Bani                                                            1
Stage 10 - Santo Domingo circuito V Centenario › Santo Domingo circuito V Centen

In [39]:
y_test = merged[(merged['name'] == 'tadej-pogacar') & (merged['year'] == 2021) & (merged['race_name'] == 'tour-de-france')]
y_test

,name,year,type,date,result,gc,icon,race_ref,race_name,race_detail,...,distance,points,Avg. speed winner:,Distance:,Parcours type:,ProfileScore:,Vert. meters:,Startlist quality score:,Won how:,adjusted_points
0,tadej-pogacar,2021,etappe,2021-07-18,72,NaN,stage,race/tour-de-france/2021/stage-21,tour-de-france,Stage 21 - Chatou › Paris Champs-Élysées,...,108.4,0.0,40.748,108.4,p1,14.0,697.0,1646,Sprint of large group,0.0
175,tadej-pogacar,2021,etappe,2021-07-17,8,1.0,chrono,race/tour-de-france/2021/stage-20,tour-de-france,Stage 20 (ITT) - Libourne › Saint-Émilion,...,30.8,26.0,51.500,30.8,p1,8.0,246.0,1646,Time Trial,342368.0
350,tadej-pogacar,2021,etappe,2021-07-16,26,1.0,stage,race/tour-de-france/2021/stage-19,tour-de-france,Stage 19 - Mourenx › Libourne,...,207.0,0.0,47.901,207.0,p2,16.0,1181.0,1646,26 km solo,0.0
528,tadej-pogacar,2021,etappe,2021-07-15,1,1.0,stage,race/tour-de-france/2021/stage-18,tour-de-france,Stage 18 - Pau › Luz Ardiden,...,129.7,50.0,36.407,129.7,p5,367.0,3561.0,1646,0.6 km solo,30204100.0
706,tadej-pogacar,2021,etappe,2021-07-14,1,1.0,stage,race/tour-de-france/2021/stage-17,tour-de-france,Stage 17 - Muret › Saint-Lary-Soulan (Col du P...,...,178.4,50.0,35.267,178.4,p5,450.0,4375.0,1646,Sprint of small group,37035000.0
886,tadej-pogacar,2021,etappe,2021-07-13,14,1.0,stage,race/tour-de-france/2021/stage-16,tour-de-france,Stage 16 - El Pas de la Casa › Saint-Gaudens,...,169.0,14.0,41.904,169.0,p4,177.0,3260.0,1646,34.6 km solo,4078788.0
1069,tadej-pogacar,2021,etappe,2021-07-11,22,1.0,stage,race/tour-de-france/2021/stage-15,tour-de-france,Stage 15 - Céret › Andorra la Vella,...,191.3,0.0,36.777,191.3,p4,234.0,4572.0,1646,19.5 km solo,0.0
1254,tadej-pogacar,2021,etappe,2021-07-10,18,1.0,stage,race/tour-de-france/2021/stage-14,tour-de-france,Stage 14 - Carcassonne › Quillan,...,183.7,6.0,43.010,183.7,p4,125.0,2877.0,1646,42.7 km solo,1234500.0
1441,tadej-pogacar,2021,etappe,2021-07-09,26,1.0,stage,race/tour-de-france/2021/stage-13,tour-de-france,Stage 13 - Nîmes › Carcassonne,...,219.9,0.0,43.332,219.9,p2,29.0,2023.0,1646,Sprint of large group,0.0
1635,tadej-pogacar,2021,etappe,2021-07-08,21,1.0,stage,race/tour-de-france/2021/stage-12,tour-de-france,Stage 12 - Saint-Paul-Trois-Châteaux › Nîmes,...,159.4,0.0,47.300,159.4,p2,52.0,2052.0,1646,11.8 km solo,0.0


In [40]:
y_test.dtypes

name                                object
year                                 int64
type                                object
date                        datetime64[ns]
result                               int64
gc                                 float64
icon                                object
race_ref                            object
race_name                           object
race_detail                         object
race_rank                           object
distance                           float64
points                             float64
Avg. speed winner:                 float64
Distance:                          float64
Parcours type:                      object
ProfileScore:                      float64
Vert. meters:                      float64
Startlist quality score:             int64
Won how:                            object
adjusted_points                    float64
dtype: object

In [42]:
min(y_test['date'])

Timestamp('2021-06-26 00:00:00')

In [41]:
min(y_test['date']) - datetime.timedelta(weeks=52)

Timestamp('2020-06-27 00:00:00')